# Optimization of Production Planning with Gurobi

Here Python and Gurobi optimization solver are utilized to optimize production planning for a factory with three production lines and reduce the total labour costs.

The input is the daily requirement in number of hours to allocate between 3 different production lines for a period of one week. Capacity is the same for all production lines but hourly cost is different as production lines 2 & 3 were added later with more modern equipment and thus more energy efficient. Due to their labour union agreement, if any of the production lines is scheduled to be open, it is required to operate for a minimum of 7 hours for that day. Also, due to the power switchs none of the production lines can operate more than 12 hours per day.

The concept of overtime and weekend extra cost is also considered in the model. Hours worked from 8 to 12 hours are paid 50% higher and hours worked during weekends are paid double. The equipment in each production line are designed for 8 hours of work per day and therefore, there is an extra cost associated for overtime which is different for each line according to their efficiency design.

Considering all of the mentioned variables, the most cost-efficient production planning will be identified using Gurobi optimization solver.



## Problem Description

As explained above, the factory possesses three production lines with more advanced technologies for more recent production lines of 2 & 3 which resulted in different operation cost for regualr hours and overtime. The hourly labor cost is 250\\$/h for lines 1, 200\\$/h for line 2, and 150\\$/h for line 3. The overtime extra cost associated for lines 1, 2 and 3 are 50\\$/h, 70\\$/h, and 90\\$/h, respectively. Due to the current rules and regulations, there are some constraints on the daily working hours: one line cannot run for less than 7 hours. Due to power switch recommendation each line can operate up to a maximum of 12 hours per day.

The requirement from customers are the number of hours of production for each day of a week, and the required hours will be scheduled such that it minimize the total cost. 

## Import necessary libraries.

In [1]:
import pandas as pd
import gurobipy
import datetime

### Input Data
The first step is to define the data that will be used for our problem, described in the problem statement. We will create some dictionaries and pandas data frame as below with the information related to customer need through the week, the timeline on which we are optimizing our planning, the list of productions lines available with the hourly cost associated.

In [2]:
needs = {
    "2020/7/13": 30,
    "2020/7/14": 10,
    "2020/7/15": 34,
    "2020/7/16": 25,
    "2020/7/17": 23,
    "2020/7/18": 24,
    "2020/7/19": 25,
}

In [3]:
reg_costs_per_line = {"Line_1": 250, "Line_2": 200, "Line_3": 150}

In [4]:
equipment_ot_costs_per_line = {"Line_1": 50, "Line_2": 70, "Line_3": 90}

In [5]:
df_prod_lines = pd.DataFrame({"reg_costs_per_line":reg_costs_per_line, 
                              'equipment_ot_costs_per_line':equipment_ot_costs_per_line})

df_prod_lines['labour_ot_costs_per_line'] = df_prod_lines['reg_costs_per_line'] *1.5
df_prod_lines['ot_costs_per_line'] = df_prod_lines['labour_ot_costs_per_line'] + df_prod_lines['equipment_ot_costs_per_line']

df_prod_lines

,reg_costs_per_line,equipment_ot_costs_per_line,labour_ot_costs_per_line,ot_costs_per_line
Line_1,250,50,375.0,425.0
Line_2,200,70,300.0,370.0
Line_3,150,90,225.0,315.0


In [6]:
ot_costs_per_line = df_prod_lines['ot_costs_per_line'].to_dict()

In [7]:
we_costs_per_line = {
    k: 2 * reg_costs_per_line[k] for k, w in reg_costs_per_line.items()
}

In [8]:
timeline = list(needs.keys())
daily_production  = list(needs.values())
prodline = list(reg_costs_per_line.keys())

n_pl = len(prodline)
n_tl = len(timeline)

As the labour cost is different during the weekend, we need to split our timeline into two lists, one for the weekdays and one for the weekend. We will later iterate the constraints on these two lists separately.

In [9]:
# Split weekdays/weekends
weekdays = []
weekend = []
for date in timeline:
    day = datetime.datetime.strptime(date, "%Y/%m/%d")
    if day.weekday() < 5:
        weekdays.append(date)
    else:
        weekend.append(date)

## Initiate the model and create the variables
We now have all the inputs defined; let us build our model. We need to initialize it and create all the variables that will be used within our function.

In [10]:
# Initiate optimization model
model = gurobipy.Model("Optimize production planning");

Restricted license - for non-production use only - expires 2022-01-13


We add with addVars() our variables and in the attributes set the size, boundaries and type for each variable.

- **total_hours**: an integer variable indexed by days and work center; it is the actual value of opening hours for each line.
- **regular hours** and **overtime hours**: an integer variable indexed by work center for each day within our scope. Boundaries are set to 7 for the lower bound and 12 for the upper bound to respect the rules and regulations of your company.
- **line_opening**: a binary variable representing the status of each production line for each day, it takes the value 0 if the line is closed and 1 if the line is running.
- **labor_cost**: an integer value measuring the daily labor cost for each line.



In [11]:
# Variable total load (hours)
total_hours = model.addVars(
    timeline,
    prodline,
    lb=0,
    ub=12,
    vtype=gurobipy.GRB.INTEGER,
    name="Total hours",
)

In [12]:
reg_hours = model.addVars(
    timeline,
    prodline,
    lb=7,
    ub=8,
    vtype=gurobipy.GRB.INTEGER,
    name="Regular hours",
)
ot_hours = model.addVars(
    timeline,
    prodline,
    lb=0,
    ub=4,
    vtype=gurobipy.GRB.INTEGER,
    name="Overtime hours",
)

In [13]:
# Status of the line ( 0 = closed, 1 = opened)
line_opening = model.addVars(
    timeline, prodline, vtype=gurobipy.GRB.BINARY, name="Open status"
)


# Variable cost
labor_cost = model.addVars(
    timeline, prodline, lb=0, vtype=gurobipy.GRB.CONTINUOUS, name="Labor cost"
)

## Set up the constraints
So far, we have just created each variable without specifying any relationships. Some variables are linked to each other, and we now must create some constraints, with addConstrs(), to set their value.

First, we need to link the variables previously created. The value of total hours is the product of working hours and line opening, and the value of labor cost is the product of total hours and the hourly cost of each work center contained in wc_reg_cost. As the labour cost is different during the weekend, we need to split our timeline into two lists, one for the weekdays and one for the weekend. 

In [14]:
# Set the value of total load (regular + overtime)
model.addConstrs(
    (
        total_hours[(date, pl)]
        == (reg_hours[(date, pl)] + ot_hours[(date, pl)]) * line_opening[(date, pl)]
        for date in timeline
        for pl in prodline
    ),
    name="Link total hours - reg/ot hours",
);

In [15]:
# Set the value of cost (hours * hourly cost)
model.addConstrs(
    (
        labor_cost[(date, pl)]
        == reg_hours[(date, pl)] * reg_costs_per_line[pl] * line_opening[(date, pl)]
        + ot_hours[(date, pl)] * ot_costs_per_line[pl] * line_opening[(date, pl)]
        for date in weekdays
        for pl in prodline
    ),
    name="Link labor cost - working hours - wd",
);


In [16]:
model.addConstrs(
    (
        labor_cost[(date, pl)]
        == total_hours[(date, pl)] * we_costs_per_line[pl]
        for date in weekend
        for pl in prodline
    ),
    name="Link labor cost - working hours - we",
);

In [17]:
model.addConstrs(
    ((total_hours.sum(date, "*") == needs[date] for date in timeline)),
    name="Link total hours - requirement",
);

## Create objective function and solution
we want to minimize our labor cost by better balancing the workload on our 3 production lines.

In [18]:
# Objective : minimize a function
model.ModelSense = gurobipy.GRB.MINIMIZE
# Function to minimize
optimization_var = gurobipy.quicksum(
    labor_cost[(date, pl)] for date in timeline for pl in prodline
)
objective = 0
objective += optimization_var

In [19]:
model.setObjective(objective)
model.optimize()

sol = pd.DataFrame(data={"Solution": model.X}, index=model.VarName)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13 rows, 105 columns and 33 nonzeros
Model fingerprint: 0xfdaeece8
Model has 36 quadratic constraints
Variable types: 21 continuous, 84 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  QMatrix range    [1e+00, 4e+02]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+01, 3e+01]
Presolve removed 3 rows and 90 columns
Presolve time: 0.01s
Presolved: 31 rows, 21 columns, 87 nonzeros
Variable types: 0 continuous, 21 integer (6 binary)
Found heuristic solution: objective 44965.000000
Found heuristic solution: objective 44900.000000
Found heuristic solution: objective 44685.000000

Root relaxation: objective 4.379847e+04, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  

## Visualizing of the results

**total_hours** was the first variable added to the model. So, it can be accessed the first.

In [20]:
total_hr = model.X[0:(n_pl * n_tl)]

line1_hr, line2_hr, line3_hr = total_hr[0::3], total_hr[1::3], total_hr[2::3]
df = pd.DataFrame({'Line1_hr': line1_hr,'Line2_hr': line2_hr,'Line3_hr': line3_hr}, index = timeline)
df['Total_hr'] = df['Line1_hr'] + df['Line2_hr'] + df['Line3_hr']
df

,Line1_hr,Line2_hr,Line3_hr,Total_hr
2020/7/13,8.0,10.0,12.0,30.0
2020/7/14,0.0,0.0,10.0,10.0
2020/7/15,10.0,12.0,12.0,34.0
2020/7/16,8.0,8.0,9.0,25.0
2020/7/17,7.0,8.0,8.0,23.0
2020/7/18,0.0,12.0,12.0,24.0
2020/7/19,7.0,7.0,11.0,25.0


**Labor cost** was the last variable added to the model. 

In [21]:
total_cost = model.X[-(n_pl * n_tl):]

line1_cost, line2_cost, line3_cost = total_cost[0::3], total_cost[1::3], total_cost[2::3]
df = pd.DataFrame({'Line1_cost': line1_cost,'Line2_cost': line2_cost,'Line3_cost': line3_cost}, index = timeline)
df['Total_cost'] = df['Line1_cost'] + df['Line2_cost'] + df['Line3_cost']
df

,Line1_cost,Line2_cost,Line3_cost,Total_cost
2020/7/13,2000.0,2340.0,2460.0,6800.0
2020/7/14,0.0,0.0,1830.0,1830.0
2020/7/15,2850.0,3080.0,2460.0,8390.0
2020/7/16,2000.0,1600.0,1515.0,5115.0
2020/7/17,1750.0,1600.0,1200.0,4550.0
2020/7/18,0.0,4800.0,3600.0,8400.0
2020/7/19,3500.0,2800.0,3300.0,9600.0


You can use the following code to visualize your model, see all your variables, constraints and the objective function, it can be helpful to correct the mistakes and review if the model is the one intended.

In [22]:
model.write("Planning_optimization.lp")
file = open("Planning_optimization.lp", 'r')
print(file.read())
file.close()

\ Model Optimize production planning
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  Labor_cost[2020/7/13,Line_1] + Labor_cost[2020/7/13,Line_2]
   + Labor_cost[2020/7/13,Line_3] + Labor_cost[2020/7/14,Line_1]
   + Labor_cost[2020/7/14,Line_2] + Labor_cost[2020/7/14,Line_3]
   + Labor_cost[2020/7/15,Line_1] + Labor_cost[2020/7/15,Line_2]
   + Labor_cost[2020/7/15,Line_3] + Labor_cost[2020/7/16,Line_1]
   + Labor_cost[2020/7/16,Line_2] + Labor_cost[2020/7/16,Line_3]
   + Labor_cost[2020/7/17,Line_1] + Labor_cost[2020/7/17,Line_2]
   + Labor_cost[2020/7/17,Line_3] + Labor_cost[2020/7/18,Line_1]
   + Labor_cost[2020/7/18,Line_2] + Labor_cost[2020/7/18,Line_3]
   + Labor_cost[2020/7/19,Line_1] + Labor_cost[2020/7/19,Line_2]
   + Labor_cost[2020/7/19,Line_3]
Subject To
 Link_labor_cost_-_working_hours_-_we[2020/7/18,Line_1]:
   - 500 Total_hours[2020/7/18,Line_1] + Labor_cost[2020/7/18,Line_1] = 0
 Link_labor_cost_-_working_hours_-_we[2020/7/18,Line